In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
keyPress = pd.read_csv('/media/josuzuki/Jo/agg_datasets/KeyPressEvent.csv',index_col = 0)

/home/josuzuki/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
keyPress = keyPress.rename(columns={'PressType':'ActivityID','ActivityID':'PressType'})

In [5]:
keyPress['SessionID'] = np.floor(keyPress.ActivityID/10e6).astype(int)

In [6]:
aux = keyPress[['Systime','SessionID']].groupby('SessionID').agg([np.min,np.max]).Systime

In [7]:
aux['diff'] = aux.amax - aux.amin
aux = aux.reset_index()

In [8]:
bla = []
step = 5000
for _,row in aux.iterrows():
    i = 0
    t0 = row.amin
    while (t0+step < row.amax):
        aux_dict = {}
        aux_dict['SessionID'] = row.SessionID
        aux_dict['WindowNumber'] = i
        aux_dict['WindowStart'] = t0
        aux_dict['WindowEnd'] = t0+step
        t0 += step
        i += 1
        bla.append(aux_dict)

In [9]:
fim = pd.DataFrame(bla)

In [56]:
fim.to_csv('skeleton.csv',index=False)

In [9]:
sk = pd.read_csv('/Users/thicorfon/Downloads/skeleton.csv')

In [11]:
sk['WindowNumber'] = np.floor(sk.WindowNumber/2)

In [17]:
grouped = sk.groupby(['SessionID','WindowNumber']).agg({
    'WindowEnd': {
        'WindowEnd':'max'
    },
    'WindowStart': {
        'WindowStart':'min'
    }
})

/Users/thicorfon/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [18]:
grouped.columns = grouped.columns.droplevel()

In [20]:
grouped = grouped.reset_index()

In [23]:
grouped['diffa'] = grouped.WindowEnd - grouped.WindowStart

In [38]:
grouped[grouped['diff'] == 10000].drop('diff',axis=1).to_csv('other_skeleton_10secs.csv',index=False)